In [1]:
import cv2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
def make_rectangle(img: np.array, js: dict) -> np.array:
    # 사각형 좌표 구하기
    top,bot = (js['top_x'],js['top_y']),(js['bot_x'],js['bot_y'])

    # 사각형 그리기
    cv2.rectangle(img,top,bot,(255,255,255)) # Grayscale 이라서 흰색으로 그려야됨
    
    return img

def otsu_algorithm(img: np.array) -> np.array:
    ret, otsu = cv2.threshold(img,-1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return otsu

In [4]:
# train valid 파일 경로
train_json_path = './dataset/train/train_defected_json'
train_img_path = './dataset/train/train_defected_dataset'

valid_json_path = './dataset/validation/validation_defected_json'
valid_img_path = './dataset/validation/validation_defected_dataset'

In [ ]:
### 이미지에 Bbox 그리기
## TrainData 

# make dir
if not os.path.isdir('./dataset/Bboximages/train'):
    os.makedirs('./dataset/Bboximages/train/st')
    os.makedirs('./dataset/Bboximages/train/aq')
    os.makedirs('./dataset/Bboximages/train/fl')

for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}')

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/train/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [13]:
### 이미지 그레이스케일로 가져와서 오츠 알고리즘 돌리기
## TrainData 

# make dir
if not os.path.isdir('./dataset/Bboximages/train'):
    os.makedirs('./dataset/Bboximages/train/st')
    os.makedirs('./dataset/Bboximages/train/aq')
    os.makedirs('./dataset/Bboximages/train/fl')

for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/train/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [18]:
### RGB 채널 쪼개서 오츠 알고리즘 적용하고 다시 합치기
## TrainData 

# make dir
if not os.path.isdir('./dataset/Otsu/train'):
    os.makedirs('./dataset/Otsu/train/st')
    os.makedirs('./dataset/Otsu/train/aq')
    os.makedirs('./dataset/Otsu/train/fl')

for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image BGR
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}')

    # RGB채널 쪼개기
    B,G,R = cv2.split(img)

    # 오츠 알고리즘 적용후 이미지 or 연산으로 합치기
    B,G,R = map(otsu_algorithm,[B,G,R])
    temp = cv2.bitwise_or(B,G)
    img = cv2.bitwise_or(temp,R)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    # cv2.imwrite(f'./dataset/Otsu/train/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [12]:
## Validation data 

# make dir
if not os.path.isdir('./dataset/Bboximages/validation'):
    os.makedirs('./dataset/Bboximages/validation/st')
    os.makedirs('./dataset/Bboximages/validation/aq')
    os.makedirs('./dataset/Bboximages/validation/fl')

for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    # apply otsu algorithm
    img = otsu_algorithm(img)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/validation/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [20]:
## Validation data 

# make dir
if not os.path.isdir('./dataset/Otsu/validation'):
    os.makedirs('./dataset/Otsu/validation/st')
    os.makedirs('./dataset/Otsu/validation/aq')
    os.makedirs('./dataset/Otsu/validation/fl')

for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image BGR
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}')

    # RGB채널 쪼개기
    B,G,R = cv2.split(img)

    # 오츠 알고리즘 적용후 이미지 or 연산으로 합치기
    B,G,R = map(otsu_algorithm,[B,G,R])
    temp = cv2.bitwise_or(B,G)
    img = cv2.bitwise_or(temp,R)

    # draw rectangle
    make_rectangle(img, json_dict)

    # save image
    cv2.imwrite(f'./dataset/Bboximages/validation/{json_dict["defect_class"]}/{json_dict["image_name"]}',img)

In [10]:
# Bbox의 모든 픽셀 값을 더해서 검출 되는지 안되는지 확인
# 이진 알고리즘 돌렸을때 검출이 된다면 Bbox 내에 반드시 흰색이 있을것이다 라는 가정
def ROI_sum(img: np.array,js: dict) -> int:
    img = img[js['top_y']:js['bot_y'], js['top_x']:js['bot_x']]
    return sum(sum(img))

valid_defected_fail = []
train_defected_fail = []

# train 데이터에서 Bbox 체크
for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image BGR
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}')

    # RGB채널 쪼개기
    B,G,R = cv2.split(img)

    # 오츠 알고리즘 적용후 이미지 or 연산으로 합치기
    B,G,R = map(otsu_algorithm,[B,G,R])
    temp = cv2.bitwise_or(B,G)
    img = cv2.bitwise_or(temp,R)

    # sum of all pixels
    pixel = ROI_sum(img, json_dict)

    # check defect
    if not pixel:
        train_defected_fail.append(idx)

# validation 데이터에서 Bbox 체크 
for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image BGR
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}')

    # RGB채널 쪼개기
    B,G,R = cv2.split(img)

    # 오츠 알고리즘 적용후 이미지 or 연산으로 합치기
    B,G,R = map(otsu_algorithm,[B,G,R])
    temp = cv2.bitwise_or(B,G)
    img = cv2.bitwise_or(temp,R)

    # sum of all pixels
    pixel = ROI_sum(img, json_dict)

    # check defect
    if not pixel:
        valid_defected_fail.append(idx)

In [13]:
fail_count = 0
for fail in train_defected_fail:
    with open(f'{train_json_path}/{fail}', 'r') as f:
        json_dict = json.load(f)
    
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)
    ret,tri = cv2.threshold(img,-1,255,cv2.THRESH_BINARY | cv2.THRESH_TRIANGLE)
    
    pixel = ROI_sum(tri, json_dict)

    if not pixel:
        fail_count += 1

for fail in valid_defected_fail:
    with open(f'{valid_json_path}/{fail}', 'r') as f:
        json_dict = json.load(f)
    
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)
    ret,tri = cv2.threshold(img,-1,255,cv2.THRESH_BINARY | cv2.THRESH_TRIANGLE)
    
    pixel = ROI_sum(tri, json_dict)

    if not pixel:
        fail_count += 1

print(fail_count/3750)

0.004


In [11]:
train_defected_fail

['seaweed_01257.json',
 'seaweed_01261.json',
 'seaweed_01282.json',
 'seaweed_01283.json',
 'seaweed_01284.json',
 'seaweed_01298.json',
 'seaweed_01302.json',
 'seaweed_01323.json',
 'seaweed_01330.json',
 'seaweed_01333.json',
 'seaweed_01353.json',
 'seaweed_01356.json',
 'seaweed_01379.json',
 'seaweed_01387.json',
 'seaweed_01394.json',
 'seaweed_01396.json',
 'seaweed_01432.json',
 'seaweed_01454.json',
 'seaweed_01498.json',
 'seaweed_01513.json',
 'seaweed_01523.json',
 'seaweed_01528.json',
 'seaweed_01557.json',
 'seaweed_01579.json',
 'seaweed_01587.json',
 'seaweed_01622.json',
 'seaweed_01631.json',
 'seaweed_01637.json',
 'seaweed_01651.json',
 'seaweed_01679.json',
 'seaweed_01682.json',
 'seaweed_01683.json',
 'seaweed_01691.json',
 'seaweed_01696.json',
 'seaweed_01697.json',
 'seaweed_01717.json',
 'seaweed_01721.json',
 'seaweed_01725.json',
 'seaweed_01742.json',
 'seaweed_01743.json',
 'seaweed_01772.json',
 'seaweed_01815.json',
 'seaweed_01838.json',
 'seaweed_0

In [19]:
defected_fail = []

for idx in os.listdir(train_json_path):
    # json files load
    with open(f'{train_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{train_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    retval, otsu_thresh = cv2.threshold(img, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 오츠 알고리즘으로 얻은 임계값을 기준으로 상한과 하한 설정
    lower_thresh = 0.5 * retval  # 하한 임계값: 오츠 임계값의 50%
    upper_thresh = retval         # 상한 임계값: 오츠 임계값 자체

    # 가우시안 블러처리
    blurred_image = cv2.GaussianBlur(img, (5, 5), 0)

    # 캐니엣지 검출
    edges = cv2.Canny(blurred_image, threshold1=lower_thresh, threshold2=upper_thresh)

    # sum of all pixels
    pixel = ROI_sum(edges, json_dict)

    # check defect
    if not pixel:
        defected_fail.append(idx)

for idx in os.listdir(valid_json_path):
    # json files load
    with open(f'{valid_json_path}/{idx}', 'r') as f:
        json_dict = json.load(f)
    
    # read image
    img = cv2.imread(f'{valid_img_path}/{json_dict["image_name"]}', cv2.IMREAD_GRAYSCALE)

    retval, otsu_thresh = cv2.threshold(img, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 오츠 알고리즘으로 얻은 임계값을 기준으로 상한과 하한 설정
    lower_thresh = 0.5 * retval  # 하한 임계값: 오츠 임계값의 50%
    upper_thresh = retval         # 상한 임계값: 오츠 임계값 자체

    # 가우시안 블러처리
    blurred_image = cv2.GaussianBlur(img, (5, 5), 0)

    # 캐니엣지 검출
    edges = cv2.Canny(blurred_image, threshold1=lower_thresh, threshold2=upper_thresh)

    # sum of all pixels
    pixel = ROI_sum(edges, json_dict)

    # check defect
    if not pixel:
        defected_fail.append(idx)

In [20]:
len(defected_fail)/3750

0.0264